In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Configurações gerais
CSV_FILE = 'C:/Users/Amanda/Desktop/projetoML/MICRODADOS_ENEM_2019.csv'
ENCODING = 'latin1'
DELIMITER = ';'

# Lista de colunas relevantes
COLUNAS_RELEVANTES = [
    'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE',
    'TP_ESCOLA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_ANO_CONCLUIU',
    'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'MEDIA_NOTAS', 'STATUS', 'Q001', 'Q002', 'Q003',
    'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013',
    'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q022', 'Q024', 'Q025'
]

def carregar_dados(file_path, encoding, delimiter):
    """Carrega o arquivo CSV em um DataFrame."""
    return pd.read_csv(file_path, encoding=encoding, delimiter=delimiter)

def preprocessar_dados(df):
    """Executa o pré-processamento dos dados."""
    # Verifica colunas que existem no DataFrame original
    colunas_existentes = [col for col in COLUNAS_RELEVANTES if col in df.columns]

    # Seleciona apenas as colunas existentes
    df = df[colunas_existentes]

    # Remove valores ausentes
    df = df.dropna()

    # Adiciona colunas calculadas, se necessário
    if all(col in df.columns for col in ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']):
        df['MEDIA_NOTAS'] = df[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)
        df['STATUS'] = df['MEDIA_NOTAS'].apply(lambda x: 'APROVADO' if x > 728.77 else 'REPROVADO')

    return df

def ajustar_questoes(df):
    """Ajusta questões de cardinalidade e substituições específicas."""
    cardinalidade_cols = ['Q001', 'Q002', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
                          'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
                          'Q017', 'Q018', 'Q019', 'Q022', 'Q024']
    no_cardinalidade_cols = ['Q003', 'Q004', 'Q025']

    for col in cardinalidade_cols:
        if 'h' in df[col].unique():
            df[col] = df[col].replace('h', 'a')  # Substitui pelo menor valor lógico

    for col in no_cardinalidade_cols:
        if 'f' in df[col].unique():
            most_frequent = df[col].mode()[0]
            df[col] = df[col].replace('f', most_frequent)

    return df

def aplicar_label_encoding(df):
    """Aplica Label Encoding nas colunas específicas."""
    label_encoder = LabelEncoder()
    label_encode_cols = ['Q001', 'Q002', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
                         'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
                         'Q017', 'Q018', 'Q019', 'Q022', 'Q024', 'TP_FAIXA_ETARIA',
                         'TP_ANO_CONCLUIU']

    for col in label_encode_cols:
        if col in df.columns:
            df[col] = label_encoder.fit_transform(df[col])

    return df

def salvar_arquivos(df, full_path, mini_path):
    """Salva os DataFrames finais em arquivos CSV."""
    df.to_csv(full_path, index=False)
    df.sample(n=10).to_csv(mini_path, index=False)

# Pipeline principal
if __name__ == "__main__":
    df = carregar_dados(CSV_FILE, ENCODING, DELIMITER)
    df = preprocessar_dados(df)
    df = ajustar_questoes(df)
    df = aplicar_label_encoding(df)
    salvar_arquivos(df, "v4.csv", "v4_mini.csv")
